In [ ]:
import yfinance as yf

In [ ]:
sp500 = yf.Ticker("^GSPC") 
# Calls the Yahoo Finance API to dowload daily stock prices
# Ticker class enables to dowload price history for single symbol from the API.

In [ ]:
sp500 = sp500.history(period="max") # Downloading the whole history of Prices.

In [ ]:
sp500.index # index and slice the database

In [ ]:
sp500.plot.line(y="Close", use_index=True)

In [ ]:
del sp500["Dividends"]
del sp500["Stock Splits"]

In [ ]:
sp500

In [ ]:
sp500["Tomorrow"] = sp500["Close"].shift(-1)

In [ ]:
sp500["Target"] = (sp500["Tomorrow"] > sp500["Close"]).astype(int)

In [ ]:
sp500 = sp500.loc["1990-01-01":].copy()

In [1]:
from sklearn.ensemble import RandomForestClassifier # RFC becuase they pickup non-linear tendencies
from sklearn.metrics import precision_score # Measuring Model Accuracy

In [ ]:
preds = model.predict(test[predictors])

In [ ]:
import pandas as pd
preds = pd.Series(preds, index=test.index)

In [ ]:
precision_score(test["Target"], preds)
combined = pd.concat([test["Target"], preds], axis=1)

In [ ]:
combined.plot()

In [ ]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Target"])
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined


In [ ]:
def backtest(data, model, predictors, start=2500, step=250):
    all_predictions = []
    
    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
    return pd.concat(all_predictions)

In [ ]:
predictions = backtest(sp500, model, predictors)
predictions["Predictions"].value_counts()

In [ ]:
precision_score(predictions["Target"], predictions["Predictions"])
predictions["Target"].value_counts() / predictions.shape[0]

In [ ]:
# Adding more predictors to our model
horizons = [2,5,60,250,1000] # The number of days
new_predictors = []

for horizon in horizons:
    rolling_averages = sp500.rolling(horizon).mean()
    
    ratio_column = f"Close_Ratio_{horizon}"
    sp500[ratio_column] = sp500["Close"] / rolling_averages["Close"]
    
    trend_column = f"Trend_{horizon}"
    sp500[trend_column] = sp500.shift(1).rolling(horizon).sum()["Target"]
    
    new_predictors += [ratio_column, trend_column]

In [ ]:
sp500 = sp500.dropna()
model = RandomForestClassifier(n_estimators=200, min_samples_split=50, random_state=1)
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Target"])
    preds = model.predict_proba(test[predictors])[:,1]
    preds[preds >= 0.6] = 1
    preds[preds < 0.6] = 0
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined

In [ ]:
predictions = backtest(sp500, model, new_predictors)
precision_score(predictions["Target"], predictions["Predictions"])
predictions["Predictions"].value_counts()